In [43]:
from pathlib import Path
from pprint import pprint
from importlib import reload

import tokenization
tokenization = reload(tokenization)

In [44]:
file = Path("/home/lk3591/Documents/code/AssemblyStyleTransfer/data/pre_normalized/0000c38f0d0bbed895f993f44d0deba00606badf225aac287fbae1e0134db72e/4298788_4298888.asm")
with open(file) as handle:
    text = handle.read()
print(text)


push ebp
mov ebp, esp
add esp, 0xfffffff4
push ebx
push esi
push edi
mov dword [var_4h], ecx
mov edi, edx
mov ebx, eax
mov al, byte [arg_8h]
call fcn.00419398
mov esi, eax
mov eax, dword [arg_ch]
push eax
mov eax, dword [var_4h]
push eax
push esi
push 0
mov eax, edi
call fcn.00404c4c
push eax                              
mov eax, dword [ebx + 4]
push eax                              
call sub.advapi32.dll_RegSetValueExA  
test eax, eax
je 0x419882
mov dword [var_ch], edi
mov byte [var_8h], 0xb                
lea eax, [var_ch]
push eax
push 0
mov ecx, dword [0x41c684]             
mov dl, 1
mov eax, dword [0x419290]             
call fcn.0040971c
call fcn.0040420c
pop edi
pop esi
pop ebx
mov esp, ebp
pop ebp
ret 8


In [45]:
normalizer = tokenization.get_normalizer()
x = normalizer.normalize_str(text)
print(x)


push ebp
mov ebp, esp
add esp, <ADR>
push ebx
push esi
push edi
mov dword [<VAR>], ecx
mov edi, edx
mov ebx, eax
mov al, byte [<ARG>]
call <FCN>
mov esi, eax
mov eax, dword [<ARG>]
push eax
mov eax, dword [<VAR>]
push eax
push esi
push 0
mov eax, edi
call <FCN>
push eax                              
mov eax, dword [ebx + 4]
push eax                              
call sub.advapi32.dll_RegSetValueExA  
test eax, eax
je <ADR>
mov dword [<VAR>], edi
mov byte [<VAR>], <ADR>                
lea eax, [<VAR>]
push eax
push 0
mov ecx, dword [<ADR>]             
mov dl, 1
mov eax, dword [<ADR>]             
call <FCN>
call <FCN>
pop edi
pop esi
pop ebx
mov esp, ebp
pop ebp
ret 8


In [46]:
pre_tokenizer = tokenization.get_pre_tokenizer()
y = pre_tokenizer.pre_tokenize_str(x)
pprint(y)

[('push', (1, 5)),
 ('ebp', (6, 9)),
 ('mov', (10, 13)),
 ('ebp', (14, 17)),
 (',', (17, 18)),
 ('esp', (19, 22)),
 ('add', (23, 26)),
 ('esp', (27, 30)),
 (',', (30, 31)),
 ('<ADR>', (32, 37)),
 ('push', (38, 42)),
 ('ebx', (43, 46)),
 ('push', (47, 51)),
 ('esi', (52, 55)),
 ('push', (56, 60)),
 ('edi', (61, 64)),
 ('mov', (65, 68)),
 ('dword', (69, 74)),
 ('[', (75, 76)),
 ('<VAR>', (76, 81)),
 (']', (81, 82)),
 (',', (82, 83)),
 ('ecx', (84, 87)),
 ('mov', (88, 91)),
 ('edi', (92, 95)),
 (',', (95, 96)),
 ('edx', (97, 100)),
 ('mov', (101, 104)),
 ('ebx', (105, 108)),
 (',', (108, 109)),
 ('eax', (110, 113)),
 ('mov', (114, 117)),
 ('al', (118, 120)),
 (',', (120, 121)),
 ('byte', (122, 126)),
 ('[', (127, 128)),
 ('<ARG>', (128, 133)),
 (']', (133, 134)),
 ('call', (135, 139)),
 ('<FCN>', (140, 145)),
 ('mov', (146, 149)),
 ('esi', (150, 153)),
 (',', (153, 154)),
 ('eax', (155, 158)),
 ('mov', (159, 162)),
 ('eax', (163, 166)),
 (',', (166, 167)),
 ('dword', (168, 173)),
 ('[', (

In [47]:
import re

pattern = r"\w+|[^\w\s]+"
string = """
    push ebp
    mov ebp, esp
    add esp, <ADR>
    push ebx
    push esi
"""

re.findall(pattern, string)

['push',
 'ebp',
 'mov',
 'ebp',
 ',',
 'esp',
 'add',
 'esp',
 ',',
 '<',
 'ADR',
 '>',
 'push',
 'ebx',
 'push',
 'esi']

In [48]:
text

'\npush ebp\nmov ebp, esp\nadd esp, 0xfffffff4\npush ebx\npush esi\npush edi\nmov dword [var_4h], ecx\nmov edi, edx\nmov ebx, eax\nmov al, byte [arg_8h]\ncall fcn.00419398\nmov esi, eax\nmov eax, dword [arg_ch]\npush eax\nmov eax, dword [var_4h]\npush eax\npush esi\npush 0\nmov eax, edi\ncall fcn.00404c4c\npush eax                              \nmov eax, dword [ebx + 4]\npush eax                              \ncall sub.advapi32.dll_RegSetValueExA  \ntest eax, eax\nje 0x419882\nmov dword [var_ch], edi\nmov byte [var_8h], 0xb                \nlea eax, [var_ch]\npush eax\npush 0\nmov ecx, dword [0x41c684]             \nmov dl, 1\nmov eax, dword [0x419290]             \ncall fcn.0040971c\ncall fcn.0040420c\npop edi\npop esi\npop ebx\nmov esp, ebp\npop ebp\nret 8'